# Ground truth neurons biological features

* author: steeve.laquitaine@epfl.ch

Setup spikeinterf_ env .. 

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm  # ground truth circuit details
import bluepy as bp  # ground truth circuit details
import spikeinterface as si
from neurom.view import plotly_impl
import neurom as nm


# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

cfg, param_conf = get_config("dense_spont", "probe_3").values()
SORTING_TRUE_PATH = cfg["ground_truth"]["full"]["output"]
BLUECFG = cfg["dataeng"]["blueconfig"]

# PARAMETERS

# SOMA
SOMA = {
    "soma": nm.core.types.NeuriteType.soma,
}
SOMA_FEATURES = [
    "radius",
    "volume",
]

# OTHER NEURITES
NEURITES_FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

DS1_MORPH = cfg["metadata"]["cell_morphs"]

2024-12-12 17:35:08,946 - root - utils.py - get_config - INFO - Reading experiment config.
2024-12-12 17:35:09,019 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Custom functions

In [2]:
def get_features(morph_paths, feature_set, neurite_set):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in neurite_set.items():
                for feature in feature_set:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df


def get_morph_paths(circuit, unit_ids: list):

    # get the units' mtypes and morphologies
    df = circuit.cells.get(unit_ids, properties=["mtype", "morphology"])

    # this creates the morphology path
    # for each unit by prefixing with the
    # common path of all morphologies
    # circuit.config["morphologies"]
    # to the morphology of the
    # units retrieved above
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-C220797A-P3_axon-sm110131a1-3_IN.. for unit 12165
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-rat_20160908_E3_LH2_cell2_axon-mt for unit 16652
    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df = df.drop(columns=["mtype", "morphology"])
    return df


def get_radius(circuit, unit_ids):

    # get the file paths of the morphology
    morph_paths = get_morph_paths(circuit, unit_ids)
    morph_paths = morph_paths.values.flatten()

    # get unit soma radius
    soma_radius = []
    for m_i in morph_paths:
        soma_radius.append(nm.load_morphology(m_i).soma.radius)
    return soma_radius

### Soma radius

In [3]:
# load once
circuit = bp.Simulation(BLUECFG).circuit

# get ground truth unit ids
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)

In [4]:
# (1m)
# get volume
radius = get_radius(circuit, SortingTrue.unit_ids)

# unit-test
assert len(radius) == len(SortingTrue.unit_ids)
print(radius)

[7.174989, 8.422994, 7.610226, 12.163964, 6.5572352, 7.9881673, 8.082553, 6.5572352, 7.275731, 8.801092, 10.477856, 6.594521, 7.610226, 9.324264, 8.801092, 8.361107, 6.5572352, 7.275731, 6.5572352, 8.082553, 7.275731, 8.422994, 7.9881673, 7.9881673, 7.9881673, 7.174989, 7.275731, 6.5572352, 7.9881673, 8.422994, 6.594521, 6.5572352, 7.610226, 9.324264, 8.361107, 8.361107, 10.56996, 10.663641, 6.5572352, 10.663641, 6.5572352, 7.047762, 6.5572352, 10.663641, 8.361107, 6.5572352, 7.610226, 8.73887, 6.5572352, 6.472128, 6.903966, 6.9807596, 7.4698386, 7.2470517, 6.896214, 9.613261, 9.613261, 7.2470517, 7.2470517, 7.368907, 7.4698386, 7.4698386, 7.368907, 6.896214, 7.4698386, 7.2470517, 7.2470517, 9.613261, 7.4698386, 6.8645296, 7.2140493, 6.5782886, 6.5782886, 6.5782886, 7.0331407, 6.23411, 7.214777, 6.23411, 7.2140493, 7.2140493, 6.5782886, 7.856283, 7.2140493, 8.047437, 6.8645296, 7.2140493, 7.0331407, 7.2140493, 8.462297, 7.2140493, 7.2140493, 6.5782886, 8.865232, 6.23411, 7.7968082, 6.8

### Neurites features

In [5]:
# 11 min

# get neurite features for each unit
df_all = pd.DataFrame()
for u_i, unit_id in enumerate(SortingTrue.unit_ids):

    # get neurite features
    morph_paths = get_morph_paths(circuit, [unit_id])
    df = get_features(morph_paths, feature_set=NEURITES_FEATURES, neurite_set=NEURITES)
    df = df.apply(lambda x: x.replace("apical", "apical_dendrite"))
    df = df.apply(lambda x: x.replace("basal", "basal_dendrite"))
    df = df.rename({"loc": "neurite_feature"}, axis="columns")
    df["neurite_feature"] += "_" + df["feature_name"]
    df.index = df.neurite_feature.tolist()
    df = df.drop(columns=["feature_name", "mtype", "neurite_feature"])
    df = df.T
    df["morph_path"] = morph_paths.iloc[0][0]
    
    # record
    df_all = pd.concat([df_all, df])

# drop index    
df_all = df_all.reset_index().drop(columns=["index"])

# concatenate with soma radius info
morph_df = pd.DataFrame()
morph_df["unit_id"] = SortingTrue.unit_ids
morph_df["soma_radius"] = radius
df = pd.concat([morph_df, df_all], axis=1)
df.to_parquet(DS1_MORPH)

mtypes: 1it [00:00,  4.92it/s]
mtypes: 1it [00:00,  2.76it/s]
mtypes: 1it [00:00,  3.13it/s]
mtypes: 1it [00:00,  1.63it/s]
mtypes: 1it [00:00,  2.67it/s]
mtypes: 1it [00:00,  1.10it/s]
mtypes: 1it [00:00,  2.07it/s]
mtypes: 1it [00:00,  3.43it/s]
mtypes: 1it [00:00,  2.64it/s]
mtypes: 1it [00:00,  1.53it/s]
mtypes: 1it [00:00,  3.02it/s]
mtypes: 1it [00:00,  1.96it/s]
mtypes: 1it [00:00,  3.84it/s]
mtypes: 1it [00:00,  3.02it/s]
mtypes: 1it [00:00,  1.52it/s]
mtypes: 1it [00:00,  2.29it/s]
mtypes: 1it [00:00,  2.43it/s]
mtypes: 1it [00:00,  3.28it/s]
mtypes: 1it [00:00,  3.34it/s]
mtypes: 1it [00:00,  4.34it/s]
mtypes: 1it [00:00,  1.82it/s]
mtypes: 1it [00:00,  3.58it/s]
mtypes: 1it [00:00,  2.93it/s]
mtypes: 1it [00:00,  3.48it/s]
mtypes: 1it [00:00,  2.01it/s]
mtypes: 1it [00:00,  1.73it/s]
mtypes: 1it [00:00,  4.05it/s]
mtypes: 1it [00:00,  2.70it/s]
mtypes: 1it [00:00,  1.51it/s]
mtypes: 1it [00:00,  2.22it/s]
mtypes: 1it [00:00,  3.72it/s]
mtypes: 1it [00:00,  2.86it/s]
mtypes: 

# References

https://neurom.readthedocs.io/en/latest/features.html#